# PPXF tests
---
This is a notebook for running tests to validate the results produced by ppxf, specifically to look at the effects of various systematics on derived star formation histories (SFHs).

### Producing a mock galaxy spectrum
---
We use the following steps to produce mock spectra to use as input to ppxf:
1. Generate a SFH by assigning weights $w_n$ to each template $T_n(\lambda)$.
2. Use the SFH to generate a mock spectrum:
    $$G(\lambda) = \sum_n w_n T_n (\lambda)$$
3. Re-bin the mock spectrum to a logarithmic wavelength grid such that 
    $$G(\lambda) \rightarrow G(\ln \lambda)$$
4. Define a line-of-sight velocity dispersion (LOSVD) where the convolution kernel is given by 
    $$\mathcal{L}(v) = \frac{1}{\sigma \sqrt{2\pi}} \exp{\frac{-(v - V)^2}{2\sigma^2}} = \frac{1}{\sigma \sqrt{2\pi}} \exp{\frac{-\Delta v^2}{2\sigma^2}} $$
where $v$ is the velocity, $V$ is the systemic velocity (assumed to be 0 for our galaxies, since we are only interested in nuclear spectra) and $\sigma$ is the LOS velocity dispersion. Define the kernel *linearly* in velocity space over a range 
    $$[-\Delta v_{\rm max}, +\Delta v_{\rm max}]$$.
5. Use the fact that, for small $\Delta v$,
    $$ \Delta v = c \Delta \ln \lambda $$ (i.e., the Doppler formula)
to re-define $\mathcal{L}(v)$ so that it is defined on a grid of constant $\Delta \ln \lambda$, so that its range is 
    $$[-(\Delta \lambda)_{\rm max}, +(\Delta \lambda)_{\rm max}] = [-\Delta v_{\rm max}/c, +\Delta v_{\rm max}/c]$$ (i.e., divide the velocity axis by $c$ in km/s). 
6. Convolve the mock spectrum with the LOSVD:
    $$ G^\prime(\ln \lambda) = G(\ln \lambda) * \mathcal{L}(\Delta \ln \lambda) $$
7. Apply the systemic redshift, i.e. "stretch" the wavelength axis:
    $$ \ln \lambda \rightarrow \ln \lambda + \ln (1 + z) $$
8. Interpolate to the linear WiFeS wavelength grid (corresponding to the "COMB" cubes).
9. Convolve by the line spread function, which we assume to be a Gaussian in wavelength space with 
    $$\sigma_{\rm LSF} = \sqrt{\sigma_{\rm WiFeS}^2 - \sigma_{\rm templates}^2}$$
10. Add noise, randomply sampled from the variance spectrum from a real WiFeS data cube.

## Sequence of tests 
---
* **Basic assurance testing**
    * Sensitivity to young stellar populations: given a young stellar population superimposed on top of an older population, what is the minimum mass fraction of the young population for which ppxf can accurately return the SFH?
    * Effect of emission lines: does the inclusion of emission lines affect the derived SFH at all? What about very broad emission lines?
    * Reducing the number of templates: by how much can we degrade the temporal & metallicity sampling of the template grid & still accurately capture the SFH? (i.e., see if the fitting process can be sped up by using fewer templates)
    * Limiting the wavelength range to save time: what happens if we use only the blue half of the spectrum?
    * Template mismatch: how accurately can the SFH be recovered when the Geneva isochrones are used to generate the mock spectra, but the Padova isochrones are used in the fitting process?
* **Regularisation**
    * Accuracy: try SFHs with varying degrees of smoothness. Using the local minimum approach to find the optimal value for regul, does ppxf accurately return the degree of smoothness?
    * If ppxf does *not* accurately capture the smoothness of the stellar continuum, what statements can we make about the accuracy of the estimated starburst age?
    * Based off these results, are there specific properties of the underlying SFH that lead to poor estimation of the regul parameter? i.e., "spiky" local minima?
* **Contamination from an AGN continuum**
    * The effect of an AGN continuum: run ppxf several times on a spectrum with varying strengths of an AGN continuum added, without making any modifications to the ppxf input. How does the strength (and slope) of the added continuum change the results? Does it have an effect on the regularisation parameter?
* **Estimating errors on the starburst age**
	* is there some kind of monte carlo method we can use to estimate errors? Or would it be better to e.g. fit a Gaussian profile to the SFH and measure a FWHM?

In [17]:
%matplotlib widget

In [41]:
# Imports
import os 
import numpy as np

from scipy import constants
from scipy.signal import convolve
from scipy.interpolate import CubicSpline

from itertools import product

import ppxf.ppxf_util as util

from cosmocalc import get_dist

import matplotlib.pyplot as plt
plt.ion()
plt.close("all")


In [19]:
# Paths 
ssp_template_path = "/home/u5708159/python/Modules/ppxftests/SSP_templates"

In [49]:
################################################################################################################
# Mock spectra options
################################################################################################################
isochrones = "Padova"  # Set of isochrones to use 
sigma_star_kms = 350   # LOS velocity dispersion, km/s
z = 0.05               # Redshift 
SNR = 25               # S/N ratio


In [50]:
################################################################################################################
# WiFeS instrument properties
################################################################################################################

# Compute the width of the LSF kernel we need to apply to the templates
FWHM_inst_A = 1.4      # for the WiFeS COMB cube; as measured using sky lines in the b3000 grating
dlambda_A_ssp = 0.30  # Gonzalez-Delgado spectra_linear have a constant spectral sampling of 0.3 A.
# Assuming that sigma = dlambda_A_ssp.
FWHM_ssp_A = 2 * np.sqrt(2 * np.log(2)) * dlambda_A_ssp
FWHM_LSF_A = np.sqrt(FWHM_inst_A**2 - FWHM_ssp_A**2)
sigma_LSF_A = FWHM_LSF_A / (2 * np.sqrt(2 * np.log(2)))

# WiFeS wavelength grid ("COMB" setting)
N_lambda_wifes = 4520
lambda_start_wifes_A = 3500.0
dlambda_wifes_A = 0.7746262160168323
lambda_vals_wifes_A = np.arange(N_lambda_wifes) * dlambda_wifes_A + lambda_start_wifes_A

oversample_factor = 4
lambda_vals_wifes_oversampled_A = np.arange(N_lambda_wifes * oversample_factor) * dlambda_wifes_A / oversample_factor + lambda_start_wifes_A

# Compute the velocity scale ("velscale") parameter from the WiFeS wavelength sampling
_, _, velscale_oversampled =\
        util.log_rebin(np.array([lambda_vals_wifes_oversampled_A[0], lambda_vals_wifes_oversampled_A[-1]]),
                       np.zeros(N_lambda_wifes * oversample_factor))

In [51]:
################################################################################################################
# Load the templates 
################################################################################################################
# List of template names - one for each metallicity
ssp_template_fnames =\
    [os.path.join(ssp_template_path, f"SSP{isochrones}", f) for f in os.listdir(os.path.join(ssp_template_path, f"SSP{isochrones}")) if f.endswith(".npz")]

################################################################################################################
# Determine how many different templates there are (i.e. N_ages x N_metallicities)
metallicities = []
ages = []
for ssp_template_fname in ssp_template_fnames:
    f = np.load(os.path.join(ssp_template_path, ssp_template_fname))
    metallicities.append(f["metallicity"].item())
    ages = f["ages"] if ages == [] else ages
    lambda_vals_ssp_linear = f["lambda_vals_A"]

# Template dimensions
N_ages = len(ages)
N_metallicities = len(metallicities)
N_lambda = len(lambda_vals_ssp_linear)

################################################################################################################
# We need to logarithmically bin 

################################################################################################################
# Load each template & store in arrays

# Create a big 3D array to hold the spectra
spec_arr_linear = np.zeros((N_metallicities, N_ages, N_lambda))

for mm, ssp_template_fname in enumerate(ssp_template_fnames):
    f = np.load(os.path.join(ssp_template_path, ssp_template_fname))
    
    # Get the spectra & wavelength values
    spectra_ssp_linear = f["L_vals"]
    lambda_vals_ssp_linear = f["lambda_vals_A"]

    # Store in the big array 
    spec_arr_linear[mm, :, :] = spectra_ssp_linear.T


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  from ipykernel import kernelapp as app


In [52]:
ssp_template_fnames

['/home/u5708159/python/Modules/ppxftests/SSP_templates/SSPPadova/SSPPadova.z004.npz',
 '/home/u5708159/python/Modules/ppxftests/SSP_templates/SSPPadova/SSPPadova.z008.npz',
 '/home/u5708159/python/Modules/ppxftests/SSP_templates/SSPPadova/SSPPadova.z019.npz']

In [53]:
################################################################################################################
# Checking to make sure the templates have been loaded in the right order! 
################################################################################################################
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 5))
# Spec 1
age_idx = 30
met_idx = 0
ax.plot(lambda_vals_ssp_linear, spec_arr_linear[met_idx, age_idx, :], label=f"t = {ages[age_idx]/1e6:.2f} Myr, met = {metallicities[met_idx]:.4f}")

# Spec 2
age_idx = 30
met_idx = -1
ax.plot(lambda_vals_ssp_linear, spec_arr_linear[met_idx, age_idx, :], label=f"t = {ages[age_idx]/1e6:.2f} Myr, met = {metallicities[met_idx]:.4f}")

ax.set_ylabel(f"$L$ (erg/s/$\AA$/M$_\odot$)")
ax.set_xlabel(f"$\lambda$")
ax.legend()
ax.autoscale(enable="True", axis="x", tight=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
def plot_sfh_mass_weighted(sfh_mass_weighted):
    # Create figure
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3.5))
    bbox = ax.get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.x0, 0.025, bbox.height])
    
    # Plot the SFH
    m = ax.imshow(np.log10(sfh_mass_weighted), cmap="magma_r", origin="lower", aspect="auto")
    fig.colorbar(m, cax=cax)
    
    # Decorations
    ax.set_yticks(range(len(metallicities)))
    ax.set_yticklabels(["{:.3f}".format(met / 0.02) for met in metallicities])
    ax.set_ylabel(r"Metallicity ($Z_\odot$)")
    cax.set_ylabel(r"Mass $\log_{10}(\rm M_\odot)$")
    ax.set_xticks(range(len(ages)))
    ax.set_xlabel("Age (Myr)")
    ax.set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical")
    
    return


In [55]:
################################################################################################################
# Define the SFH 
################################################################################################################
# Idea 1: use a Gaussian kernel to smooth "delta-function"-like SFHs
# Idea 2: are the templates logarithmically spaced in age? If so, could use e.g. every 2nd template 

sfh_mass_weighted = np.zeros((N_metallicities, N_ages))

age_1 = 10e6  # yr 
age_2 = 10e9  # yr

age_1_idx = np.nanargmin(np.abs(ages - age_1))
age_2_idx = np.nanargmin(np.abs(ages - age_2))

met_idx = 1  # Solar metalliciy for now 

sfh_mass_weighted[met_idx, age_1_idx] = 1e7   # solar masses
sfh_mass_weighted[met_idx, age_2_idx] = 1e10  # solar masses

# Plot to check
plot_sfh_mass_weighted(sfh_mass_weighted)
plt.gcf().get_axes()[0].set_title("Input SFH")

# TODO: need to figure out how to get these back from the ppxf results...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log10
  


Text(0.5, 1.0, 'Input SFH')

In [56]:
spec_arr_linear.shape

(3, 74, 13321)

In [57]:
################################################################################################################
# Create the mock spectrum
################################################################################################################
# Some settings for plotting
fig_w = 12
fig_h = 5
lambda_1 = 5800
lambda_2 = 6100

# 1. Sum the templates by their weights to create a single spectrum
spec_linear = np.nansum(np.nansum(sfh_mass_weighted[:, :, None] * spec_arr_linear, axis=0), axis=0)

# Plot to check
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h))
ax.plot(lambda_vals_ssp_linear, spec_linear, color="black", label="Spectrum")
for mm, aa in product(range(N_metallicities), range(N_ages)):
    w = sfh_mass_weighted[mm, aa]
    if w > 0:
        ax.plot(lambda_vals_ssp_linear, spec_arr_linear[mm, aa, :] * w, 
                label=f"t = {ages[aa] / 1e6:.2f} Myr, m = {metallicities[mm]:.4f}, w = {w:g}")
ax.set_ylabel(f"$L$ (erg/s/$\AA$/M$_\odot$)")
ax.set_xlabel(f"$\lambda$")
ax.legend()
ax.autoscale(enable="True", axis="x", tight=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
################################################################################################################
# 2. Logarithmically re-bin
spec_log, lambda_vals_ssp_log, velscale_temp = util.log_rebin(
    np.array([lambda_vals_ssp_linear[0], lambda_vals_ssp_linear[-1]]),
    spec_linear, velscale=velscale_oversampled)

# Plot to check
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h))
ax.plot(lambda_vals_ssp_linear, spec_linear, color="black", label="Normalised, linear spectrum")
ax.plot(np.exp(lambda_vals_ssp_log), spec_log, color="red", label="Normalised, logarithmically-binned spectrum")

ax.set_ylabel(f"$L$ + offset (normalised)")
ax.set_xlabel(f"$\lambda$")
ax.legend()
ax.autoscale(enable="True", axis="x", tight=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
################################################################################################################
# 3a. Create the kernel corresponding to the LOSVD
delta_lnlambda = np.diff(lambda_vals_ssp_log)[0]
delta_lnlambda_vals = (np.arange(400) - 200) * delta_lnlambda

# 3b. convert the x-axis to units of delta v (km/s) by multiplying by c (in km/s)
c_kms = constants.c / 1e3
delta_v_vals_kms = delta_lnlambda_vals * c_kms
kernel_losvd = 1 / (np.sqrt(2 * np.pi) * sigma_star_kms) *\
         np.exp(- (delta_v_vals_kms**2) / (2 * sigma_star_kms**2))

# Plot to check
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.plot(delta_v_vals_kms, kernel_losvd)
ax.axvline(0, color="black")
ax.set_xlabel(r"$\Delta v$")

fig, ax = plt.subplots(nrows=1, ncols=1)
ax.plot(delta_lnlambda_vals, kernel_losvd)
ax.axvline(0, color="black")
ax.set_xlabel(r"$\Delta ln \lambda $")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, '$\\Delta ln \\lambda $')

In [60]:
################################################################################################################
# 4. Convolve the LOSVD kernel with the mock spectrum
spec_log_conv = convolve(spec_log, kernel_losvd, mode="same") / np.nansum(kernel_losvd)

# Plot to check
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h))
ax.plot(np.exp(lambda_vals_ssp_log), spec_log, color="black", label="Before convolution with LOSV")
ax.plot(np.exp(lambda_vals_ssp_log), spec_log_conv, color="red", label="After convolution with LOSVD")

ax.set_ylabel(f"$L$")
ax.set_xlabel(f"$\lambda$")
ax.legend()
ax.autoscale(enable="True", axis="x", tight=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
################################################################################################################
# 5. Apply the redshift 
lambda_vals_ssp_log_redshifted = lambda_vals_ssp_log + np.log(1 + z)

# Plot to check
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h))
ax.plot(np.exp(lambda_vals_ssp_log), spec_log_conv, color="black", label="Before redshifting")
ax.plot(np.exp(lambda_vals_ssp_log_redshifted), spec_log_conv, color="red", label="After redshifting")
    
ax.set_ylabel(f"$L$")
ax.set_xlabel(f"$\lambda$")
ax.legend()
ax.autoscale(enable="True", axis="x", tight=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
################################################################################################################
# 6. Interpolate to the WiFeS wavelength grid (corresponding to the COMB data cube) using a cubic spline
cs = CubicSpline(np.exp(lambda_vals_ssp_log_redshifted), spec_log_conv)
spec_wifes_conv = cs(lambda_vals_wifes_oversampled_A)

# Plot to check
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h))
ax.step(np.exp(lambda_vals_ssp_log_redshifted), spec_log_conv, color="black", label="Before interpolation", where="mid")
ax.step(lambda_vals_wifes_oversampled_A, spec_wifes_conv, color="red", label="Interpolated to WiFeS wavelength grid", where="mid")

ax.legend() 
ax.set_xlabel(f"$\lambda$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, '$\\lambda$')

In [63]:
################################################################################################################
# 7. Convolve by the line spread function
lambda_vals_lsf_oversampled_A = (np.arange(100) - 50) * dlambda_wifes_A / 4
kernel_lsf = np.exp(- (lambda_vals_lsf_oversampled_A**2) / (2 * sigma_LSF_A**2))

spec_wifes_conv_lsf = convolve(spec_wifes_conv, kernel_lsf, mode="same") / np.nansum(kernel_lsf)

# Plot to check
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h))
ax.step(lambda_vals_wifes_oversampled_A, spec_wifes_conv, color="black", label="Before convolution with LSF", where="mid")
ax.step(lambda_vals_wifes_oversampled_A, spec_wifes_conv_lsf, color="red", label="After convolution with LSF", where="mid")

ax.legend() 
ax.set_xlabel(f"$\lambda$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, '$\\lambda$')

In [64]:
################################################################################################################
# 8. Downsample to the WiFeS wavelength grid (corresponding to the COMB data cube)
spec_wifes = np.nansum(spec_wifes_conv_lsf.reshape(-1, oversample_factor), axis=1) / oversample_factor

# Plot to check
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h))
ax.step(lambda_vals_wifes_oversampled_A, spec_wifes_conv, color="black", label="Before downsampling", where="mid")
ax.step(lambda_vals_wifes_A, spec_wifes, color="red", label="After downsampling", where="mid")
ax.set_xlim([lambda_1 + 20, lambda_2 - 20])

ax.legend() 
ax.set_xlabel(f"$\lambda$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, '$\\lambda$')

In [65]:
################################################################################################################
# Convert to units of erg/s/cm2/A
D_A_Mpc, D_L_Mpc = get_dist(z, H0=70.0, WM=0.3)
D_L_cm = D_L_Mpc * 1e6 * 3.086e18
spec_wifes_flambda = spec_wifes * 1 / (4 * np.pi * D_L_cm**2)


In [67]:
################################################################################################################
# 9. Add noise. 
spec_wifes_flambda_err = spec_wifes_flambda / SNR
noise = np.random.normal(loc=0, scale=spec_wifes_flambda_err)
spec_wifes_noisy = spec_wifes_flambda + noise

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h))
ax.step(lambda_vals_wifes_A, noise, color="black", label="Noise", where="mid")


# Plot to check
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h))
ax.step(lambda_vals_wifes_A, spec_wifes_flambda, color="black", label="Before noise", where="mid")
ax.step(lambda_vals_wifes_A, spec_wifes_noisy + 2e-17, color="red", label="After noise", where="mid")

ax.legend() 
ax.set_xlabel(f"$\lambda$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, '$\\lambda$')

In [78]:
################################################################################################################
# Run ppxf
################################################################################################################
from ppxftests.run_ppxf import run_ppxf

run_ppxf(spec=spec_wifes_noisy, spec_err=spec_wifes_flambda_err, lambda_vals_A=lambda_vals_wifes_A,
         FWHM_inst_A=FWHM_inst_A, z=z, ngascomponents=1, isochrones="Padova",
         tie_balmer=True)

Median SNR = 25.0036
Emission lines included in gas templates:
['Balmer' '[OII]3726' '[OII]3729' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d' '[NeIII]3869' 'HeI3889']
Best Fit:       Vel     sigma
 comp. 0:     14640       336
 comp. 1:     14586        65
chi2/DOF: 1.073
method = capfit ; Jac calls: 5 ; Func calls: 52 ; Status: 2
Gas Reddening E(B-V): 0.000
Nonzero Templates:  13  /  224
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp: 1  Balmer (component 1)    0.09705      0.12   14586    65
Comp: 1  [OII]3726 (component 1)     0.1098     0.052   14586    65
Comp: 1  [OII]3729 (component 1)          0     0.052   14586    65
Comp: 1  [OIII]5007_d (component 1)      0.228       0.1   14586    65
Comp: 1  [OI]6300_d (component 1)          0     0.091   14586    65
Comp: 1  [NII]6583_d (component 1)          0     0.088   14586    65
Comp: 1  [NeIII]3869 (comp

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'weights_mass_weighted_metallicity_summed' is not defined